In [1]:
#run this line if you need to re-install graph data science in a fresh environment
#!pip install graphdatascience

In [2]:
from graphdatascience import GraphDataScience

In [3]:
gds = GraphDataScience('bolt://3.86.239.25:7687', auth=('neo4j', 'canisters-amplifiers-bets'))

## Native Projection Basics

In [4]:
# basic homogeneous directed projection
g, _  = gds.graph.project('directedProj', 'Airport', 'HAS_ROUTE')

In [5]:
# view output
_

nodeProjection            {'Airport': {'label': 'Airport', 'properties':...
relationshipProjection    {'HAS_ROUTE': {'orientation': 'NATURAL', 'aggr...
graphName                                                      directedProj
nodeCount                                                              3503
relationshipCount                                                     46389
projectMillis                                                           136
Name: 0, dtype: object

In [6]:
# undirected homogeneous projection
g, _  = gds.graph.project('undirectedProj', 'Airport',
                          {'HAS_ROUTE': {'orientation': 'UNDIRECTED'}})
_

nodeProjection            {'Airport': {'label': 'Airport', 'properties':...
relationshipProjection    {'HAS_ROUTE': {'orientation': 'UNDIRECTED', 'a...
graphName                                                    undirectedProj
nodeCount                                                              3503
relationshipCount                                                     92778
projectMillis                                                           272
Name: 0, dtype: object

In [7]:
# create directed projection with relationship weights
g, _  = gds.graph.project('weightedProj', 'Airport', 'HAS_ROUTE',
                          relationshipProperties=['distance'])
_

nodeProjection            {'Airport': {'label': 'Airport', 'properties':...
relationshipProjection    {'HAS_ROUTE': {'orientation': 'NATURAL', 'aggr...
graphName                                                      weightedProj
nodeCount                                                              3503
relationshipCount                                                     46389
projectMillis                                                           208
Name: 0, dtype: object

In [8]:
# create projection with multiple nodes and relationships
g, _  = gds.graph.project('heterogeneousProj', ['Airport', 'City'],
    {
        'HAS_ROUTE': {
            'orientation': 'NATURAL',
            'properties': 'distance'
        },
        'IN_CITY': {
            'orientation': 'UNDIRECTED',
        },
    })
_

nodeProjection            {'Airport': {'label': 'Airport', 'properties':...
relationshipProjection    {'HAS_ROUTE': {'orientation': 'NATURAL', 'aggr...
graphName                                                 heterogeneousProj
nodeCount                                                              6862
relationshipCount                                                     53395
projectMillis                                                           418
Name: 0, dtype: object

## Other Native Projection Features

We covered the basics here but there are actually many extended syntax and configuration options available for native projections

__Documentation__: [https://neo4j.com/docs/graph-data-science/current/management-ops/projections/graph-project/](https://neo4j.com/docs/graph-data-science/current/management-ops/projections/graph-project/)

Other capabilities include:
- reverse relationship orientation
- node properties
- parallel relationship aggregations

 In general, if you are trying to do something with native projection and can’t quite express the thing you want, check the docs to see if there are additional configurations or syntax extensions to support.

# Cleanup

In [9]:
# drop projection with current reference
g.drop()

graphName                                            heterogeneousProj
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                         6862
relationshipCount                                                53395
configuration        {'relationshipProjection': {'HAS_ROUTE': {'ori...
density                                                       0.001134
creationTime                       2022-11-15T14:11:00.190727000+00:00
modificationTime                   2022-11-15T14:11:00.628419000+00:00
schema               {'graphProperties': {}, 'relationships': {'HAS...
Name: 0, dtype: object

In [10]:
# get all projected graphs on the server that we created
graphs = gds.graph.list()
graphs

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,"{'p99': 175, 'min': 0, 'max': 307, 'mean': 13....",weightedProj,neo4j,7043 KiB,7212552,3503,46389,{},0.003781,2022-11-15T14:10:59.930340000+00:00,2022-11-15T14:11:00.139571000+00:00,"{'graphProperties': {}, 'relationships': {'HAS..."
1,"{'p99': 334, 'min': 0, 'max': 610, 'mean': 26....",undirectedProj,neo4j,1127 KiB,1154104,3503,92778,{'relationshipProjection': {'HAS_ROUTE': {'ori...,0.007563,2022-11-15T14:10:59.560573000+00:00,2022-11-15T14:10:59.826917000+00:00,"{'graphProperties': {}, 'relationships': {'HAS..."
2,"{'p99': 175, 'min': 0, 'max': 307, 'mean': 13....",directedProj,neo4j,871 KiB,891936,3503,46389,{'relationshipProjection': {'HAS_ROUTE': {'ori...,0.003781,2022-11-15T14:10:59.376864000+00:00,2022-11-15T14:10:59.450023000+00:00,"{'graphProperties': {}, 'relationships': {'HAS..."


In [11]:
# drop all graph projections
g_names = graphs.graphName.tolist()
for g_name in g_names:
    g = gds.graph.get(g_name)
    gds.graph.drop(g)

In [12]:
gds.graph.list()

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
